<a href="https://colab.research.google.com/github/luciasalmeron/TFM_Ingenieria_Biomedica_y_Salud_Digital/blob/main/Evaluacion_de_modelos_balanceo_de_clases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Montaje de Google Drive y Librerías

In [ ]:
from google.colab import drive

# Monta Google Drive
drive.mount('/content/drive')

# https://api.isic-archive.com/collections/65/

Mounted at /content/drive


In [ ]:
!pip install tensorflow
import os
import pandas as pd
import shutil
from PIL import Image
import matplotlib.pyplot as plt

!pip install split-folders
import splitfolders
import tensorflow as tf # This line was causing the error
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
import time
import zipfile



#2. Selección del modelo


In [ ]:
## SELECT THE DESIRED MODEL ARCHITECTURE ##
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2 as model, preprocess_input
#from tensorflow.keras.applications.resnet50 import ResNet50 as model, preprocess_input
#from tensorflow.keras.applications.vgg16 import VGG16 as model, preprocess_input
#from tensorflow.keras.applications.inception_v3 import InceptionV3 as model, preprocess_input
##from tensorflow.keras.applications.densenet121 import DenseNet121 as model, preprocess_input ME DA ERROR AL EJECUTARLO
#from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2L as model, preprocess_input
#from tensorflow.keras.applications.xception import Xception as model, preprocess_input
#from tensorflow.keras.applications.convnext import ConvNeXtLarge as model, preprocess_input

from tensorflow.keras.metrics import Accuracy, AUC, Precision, Recall, TruePositives, TrueNegatives, FalsePositives, FalseNegatives


#3. Definición de Parámetros

In [ ]:
NUM_CLASSES = 2
CLASSES = ['Benign', 'Malignant']
##MALIGNANT -->  POSITIVA
##BENIGN --> NEGATIVA

## Training variables ##
INPUT_SIZE = 224
BATCH_SIZE = 32  # size of the readed batches from generator, must fit on memory
VAL_SPLIT = 0.15  # fraction of the images used for validation
TEST_SPLIT = 0.15  # fraction of the images used for testing
EPOCHS = 20



image_folder = '/content/imagenes520/imagenes520'
#image_files = [f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))]


# Crear las carpetas "benigno" y "maligno" si no existen
BASE_DATASET = '/content/imagenes520/imagenes520/imagenes_clasificadas'   # Carpeta base con las carpetas "benigno" y "maligno"
benigno_folder = BASE_DATASET+'/clasificadas_benigno/'
maligno_folder = BASE_DATASET+'/clasificadas_maligno/'

SPLITTED_DATASET = 'splitted_dataset'  # Carpeta para los datos divididos
SAVE_MODELS_PATH = '/content/drive/My Drive/trained_models'  # Carpeta para guardar los modelos




#4. Extracción de las Imágenes y Clasificación por Carpetas


In [ ]:
!cp "/content/drive/My Drive/imagenes520.zip" "/content/imagenes520.zip"

In [ ]:

zip_path = "/content/imagenes520.zip"  # Ruta del archivo ZIP
extract_path = "/content/imagenes520"  # Carpeta de destino

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)



In [ ]:

# Ruta a la carpeta de imágenes en Google Drive

# Lista los archivos en la carpeta
print(f"Se encontraron {len(image_files)} imágenes.")
# Mostrar las primeras 5 imágenes
for img_file in image_files[:5]:
    img_path = os.path.join(image_folder, img_file)
    img = Image.open(img_path)
    plt.imshow(img)
    plt.title(img_file)
    plt.axis('off')
    plt.show()

'\n# Ruta a la carpeta de imágenes en Google Drive\n\n# Lista los archivos en la carpeta\nprint(f"Se encontraron {len(image_files)} imágenes.")\n\n# Mostrar las primeras 5 imágenes\n#for img_file in image_files[:5]:\n#    img_path = os.path.join(image_folder, img_file)\n#    img = Image.open(img_path)\n#    plt.imshow(img)\n#    plt.title(img_file)\n#    plt.axis(\'off\')\n#    plt.show()'

#5. Clasificación de Imágenes en Benigno y Maligno

In [ ]:
# Leer el archivo Excel
df = pd.read_csv("/content/drive/My Drive/challenge-training_metadata.csv", sep=";")


# Crear las carpetas si no existen
os.makedirs(benigno_folder, exist_ok=True)
os.makedirs(maligno_folder, exist_ok=True)

# Contadores
benigno_count = 0
maligno_count = 0
no_encontrado_count = 0

# Clasificar las imágenes
for index, row in df.iterrows():
    image_name = row['isic_id'] + '.jpg'
    label = row['diagnosis_1']
    image_path = os.path.join(image_folder, image_name)

    if os.path.exists(image_path):
        if label == 'Benign':
            shutil.move(image_path, os.path.join(benigno_folder, image_name))
            benigno_count += 1
        elif label == 'Malignant':
            shutil.move(image_path, os.path.join(maligno_folder, image_name))
            maligno_count += 1
    else:
        #print(f"Imagen no encontrada: {image_name}")
        no_encontrado_count += 1

    # Mostrar progreso en tiempo real
    print(f"Movidas - Benigno: {benigno_count}, Maligno: {maligno_count}, No encontradas: {no_encontrado_count}")

# Resumen final
print("\nProceso finalizado.")
print(f"Total movidas a Benigno: {benigno_count}")
print(f"Total movidas a Maligno: {maligno_count}")
print(f"Total imágenes no encontradas: {no_encontrado_count}")


Se han truncado las últimas 5000 líneas del flujo de salida.
Movidas - Benigno: 46250, Maligno: 6648, No encontradas: 0
Movidas - Benigno: 46250, Maligno: 6649, No encontradas: 0
Movidas - Benigno: 46251, Maligno: 6649, No encontradas: 0
Movidas - Benigno: 46251, Maligno: 6650, No encontradas: 0
Movidas - Benigno: 46251, Maligno: 6651, No encontradas: 0
Movidas - Benigno: 46252, Maligno: 6651, No encontradas: 0
Movidas - Benigno: 46253, Maligno: 6651, No encontradas: 0
Movidas - Benigno: 46254, Maligno: 6651, No encontradas: 0
Movidas - Benigno: 46254, Maligno: 6652, No encontradas: 0
Movidas - Benigno: 46254, Maligno: 6653, No encontradas: 0
Movidas - Benigno: 46255, Maligno: 6653, No encontradas: 0
Movidas - Benigno: 46256, Maligno: 6653, No encontradas: 0
Movidas - Benigno: 46256, Maligno: 6654, No encontradas: 0
Movidas - Benigno: 46257, Maligno: 6654, No encontradas: 0
Movidas - Benigno: 46257, Maligno: 6655, No encontradas: 0
Movidas - Benigno: 46258, Maligno: 6655, No encontrada

#6. Comprobacion imagenes por carpeta

In [ ]:

# Contar las imágenes en cada carpeta
benigno_count = len([f for f in os.listdir(benigno_folder) if os.path.isfile(os.path.join(benigno_folder, f))])
maligno_count = len([f for f in os.listdir(maligno_folder) if os.path.isfile(os.path.join(maligno_folder, f))])

# Mostrar los resultados
print(f"Número de imágenes en la carpeta 'benigno': {benigno_count}")
print(f"Número de imágenes en la carpeta 'maligno': {maligno_count}")

# Contar las imágenes en la carpeta original
original_count = len([f for f in os.listdir(image_folder) if os.path.isfile(os.path.join(image_folder, f))])

# Mostrar el resultado
print(f"Número de imágenes en la carpeta original 'image_folder': {original_count}")


Número de imágenes en la carpeta 'benigno': 48529
Número de imágenes en la carpeta 'maligno': 9057
Número de imágenes en la carpeta original 'image_folder': 20093


#7. División en Entrenamiento, Validación y Test

In [ ]:
# Dividir el dataset en entrenamiento, validación y prueba
if not os.path.exists(SPLITTED_DATASET):
    splitfolders.ratio(BASE_DATASET, output=SPLITTED_DATASET, seed=123, ratio=(1 - VAL_SPLIT - TEST_SPLIT, VAL_SPLIT, TEST_SPLIT))



Copying files: 57586 files [00:10, 5546.03 files/s]


#8. Cálculo de Pesos de Clases (Balanceo)

In [ ]:
def count_images_in_folders(base_path):
    folder_counts = {}
    for folder in os.listdir(base_path):
        folder_path = os.path.join(base_path, folder)
        if os.path.isdir(folder_path):
            num_images = len([f for f in os.listdir(folder_path) if f.lower().endswith(('png', 'jpg', 'jpeg', 'bmp', 'gif'))])
            folder_counts[folder] = num_images
    return folder_counts

training_path = 'splitted_dataset/train'
validation_path = 'splitted_dataset/val'
test_path = 'splitted_dataset/test'

training_counts = count_images_in_folders(training_path)
validation_counts = count_images_in_folders(validation_path)
test_counts = count_images_in_folders(test_path)

from sklearn.utils.class_weight import compute_class_weight
import numpy as np
# Definir clases
classes = np.array([0, 1])  # 0 = Benigno, 1 = Maligno


num_benign = training_counts["clasificadas_benigno"]
num_malignant = training_counts["clasificadas_maligno"]

# Calcular los pesos de cada clase
class_weights = compute_class_weight(class_weight="balanced", classes=classes,
                                     y=np.concatenate([np.zeros(num_benign), np.ones(num_malignant)]))

# Convertir a diccionario
class_weights = {i: weight for i, weight in enumerate(class_weights)}

print("Pesos de clase:", class_weights)

print("Training set image counts:")
for folder, count in training_counts.items():
    print(f"{folder}: {count} images")

print("\nValidation set image counts:")
for folder, count in validation_counts.items():
    print(f"{folder}: {count} images")

print("\nTest set image counts:")
for folder, count in test_counts.items():
    print(f"{folder}: {count} images")

Pesos de clase: {0: np.float64(0.593302914336179), 1: np.float64(3.1794447073670926)}
Training set image counts:
clasificadas_benigno: 33970 images
clasificadas_maligno: 6339 images

Validation set image counts:
clasificadas_benigno: 7279 images
clasificadas_maligno: 1358 images

Test set image counts:
clasificadas_benigno: 7280 images
clasificadas_maligno: 1360 images


#9. Generadores de Datos con Preprocesamiento

In [ ]:
# Generadores de imágenes
#datagen = ImageDataGenerator(rescale=1.0/255)

def preprocess(images):
      # Using the preprocess function of the selected model
      # To ensure the new data is in the same format as the original data the model was trained on
      return preprocess_input(images)




seed=123

train_datagen = ImageDataGenerator(#rescale=1./255,
                                    #rotation_range = 25,
                                    #shear_range=0.2,
                                    #zoom_range=0.2,
                                    #width_shift_range=0.0,
                                    #height_shift_range=0.0,
                                    #brightness_range=None,
                                    #horizontal_flip=True,
                                    #vertical_flip=True,
                                    fill_mode='wrap',
                                    preprocessing_function=preprocess)

train_generator = train_datagen.flow_from_directory(
    training_path,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

validation_generator = train_datagen.flow_from_directory(
    validation_path,
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary'
)

test_datagen = ImageDataGenerator(
    fill_mode='wrap',
    preprocessing_function=preprocess
)

test_generator = test_datagen.flow_from_directory(
    test_path,  # ← asegúrate de definir esta variable correctamente
    target_size=(INPUT_SIZE, INPUT_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='binary',
    shuffle=False  # Muy importante para evaluar correctamente
)


Found 40309 images belonging to 2 classes.
Found 8637 images belonging to 2 classes.
Found 8640 images belonging to 2 classes.


#10. Comprobar que el generador funciona correctamente

In [ ]:
images, labels = train_generator.__next__()
print('Images is shape', images.shape)

for i in range(4):
    f = plt.figure(figsize = (20, 20))
    f.add_subplot(1,3,1)
    plt.axis('off')
    plt. title('Label: '+str(CLASSES[int(labels[i])]))
    #plt.imshow(images[i])
    plt.imshow(images[i].astype('uint8'),vmin=0, vmax=1)

plt.show(block=True)
plt.show

"images, labels = train_generator.__next__()\nprint('Images is shape', images.shape)\n\nfor i in range(4):\n    f = plt.figure(figsize = (20, 20))\n    f.add_subplot(1,3,1)\n    plt.axis('off')\n    plt. title('Label: '+str(CLASSES[int(labels[i])]))\n    #plt.imshow(images[i])\n    plt.imshow(images[i].astype('uint8'),vmin=0, vmax=1)\n\nplt.show(block=True)\nplt.show"

#11. Generacion de la red neuronal


In [ ]:
'''##########          CNN Construction           ##########'''

INPUT_SHAPE = (INPUT_SIZE, INPUT_SIZE, 3)

base_model = model(input_shape=INPUT_SHAPE,
                   include_top=False,
                   weights='imagenet')


# Guarda el nombre del modelo en una variable
model_name = model.__name__


base_model.trainable = False

model = tf.keras.Sequential([
                          base_model,
                          tf.keras.layers.GlobalAveragePooling2D(),
                          tf.keras.layers.Dense(1, activation='sigmoid')
                        ])


metrics = ['accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives']

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=metrics)



9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


#12. Entrenamiento con Callbacks

In [ ]:
# Callbacks

early_stop = EarlyStopping(monitor='val_accuracy', patience=3, restore_best_weights=True)

# Entrenar el modelo

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    class_weight=class_weights,  # Aplicamos pesos de clases
    validation_data=validation_generator,
    callbacks=[early_stop]
)


# Evaluar el modelo
#test_loss, test<_accuracy = model.evaluate(test_generator)
#print(f"Accuracy en el conjunto de prueba: {test_accuracy}")


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/20
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 830s 654ms/step - AUC: 0.8772 - FalseNegatives: 599.1309 - FalsePositives: 3046.4385 - Precision: 0.4418 - Recall: 0.7941 - TrueNegatives: 13944.9307 - TruePositives: 2595.6646 - accuracy: 0.8057 - loss: 0.4378 - val_AUC: 0.9184 - val_FalseNegatives: 191.0000 - val_FalsePositives: 1262.0000 - val_Precision: 0.4804 - val_Recall: 0.8594 - val_TrueNegatives: 6017.0000 - val_TruePositives: 1167.0000 - val_accuracy: 0.8318 - val_loss: 0.3823
Epoch 2/20
1260/1260 ━━━━━━━━━━━━━━━━━━━━ 815s 647ms/step - AUC: 0.9178 - FalseNegatives: 482.1388 - FalsePositives: 2801.3665 - Precision: 0.4935 - Recall: 0.8503 - TrueNegatives: 14202.1709 - TruePositives: 2699.3728 - accuracy: 0.8384 - loss: 0.3653 - val_AUC: 0.9252 - val_FalseNegatives: 152.0000 - val_FalsePositives: 1376.0000 - val_Precision: 0.4671 - val_Recall: 0.8881 - val_TrueNegatives: 5903.0000 - val_TruePositives: 1206.0000 - val_accuracy: 0.8231 - val_loss: 0.4194
Epoch 3/20
1260/1260 ━━━━━━━━━━━━━

#13. Guardado del Modelo y del Histórico

---



In [ ]:
# Guardar el modelo
from datetime import datetime

# Obtener la mejor precisión de validación
best_val_accuracy = max(history.history['val_accuracy'])  # Mejor val_accuracy alcanzada
best_val_accuracy_str = f"{best_val_accuracy:.4f}"  # Formatear 4 decimales

# Obtener timestamp sin espacios
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

# Crear el nombre del archivo con timestamp y val_accuracy
filename = f"Modelo_{model_name}_{timestamp}_valacc{best_val_accuracy_str}.h5"

# Crear carpetas para modelos y logs
os.makedirs(SAVE_MODELS_PATH, exist_ok=True)
model.save(os.path.join(SAVE_MODELS_PATH, filename))

# Convertir el history a DataFrame
history_df = pd.DataFrame(history.history)

# Guardar en un CSV
history_filename = f"History_{model_name}_{timestamp}_valacc{best_val_accuracy_str}.csv"
history_df.to_csv(os.path.join(SAVE_MODELS_PATH, history_filename), index=False)



#14. Evaluacion con el conjunto de test

In [ ]:
##########           Model test           ##########

# Evaluate on Validation data
scores = model.evaluate(test_generator)
for name, value in zip(metrics, scores):
  print(name, ': ', value)


270/270 ━━━━━━━━━━━━━━━━━━━━ 132s 486ms/step - AUC: 0.1450 - FalseNegatives: 18.1439 - FalsePositives: 935.8229 - Precision: 0.0478 - Recall: 0.1298 - TrueNegatives: 3301.8745 - TruePositives: 96.0406 - accuracy: 0.7522 - loss: 0.5389
accuracy :  0.39487749338150024
AUC :  0.8302083611488342
Precision :  0.9081038236618042
Recall :  0.47753044962882996
TruePositives :  0.8360294103622437
TrueNegatives :  1137.0
FalsePositives :  6036.0
FalseNegatives :  1244.0


In [ ]:
#metrics = ['loss','accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives']


from tensorflow.keras.models import load_model
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns


# Cargar el modelo
Modelo_MobileNetV2 = load_model('/content/drive/MyDrive/trained_models/Modelos_balanceo_520/Modelo_MobileNetV2_20250318_214126_valacc0.8813.h5')
metrics = ['loss','accuracy', 'AUC', 'Precision', 'Recall', 'TruePositives', 'TrueNegatives', 'FalsePositives', 'FalseNegatives']

# Evaluate on Validation data
scores = Modelo_MobileNetV2.evaluate(test_generator)
for name, value in zip(metrics, scores):
  print(name, ': ', value)



270/270 ━━━━━━━━━━━━━━━━━━━━ 76s 280ms/step - accuracy: 0.8576 - auc_3: 0.1465 - false_negatives_3: 31.8561 - false_positives_3: 534.0701 - loss: 0.3294 - precision_3: 0.0609 - recall_3: 0.1062 - true_negatives_3: 3703.6272 - true_positives_3: 82.3284
loss :  0.28866466879844666
accuracy :  0.8791666626930237
AUC :  0.9149766564369202
Precision :  0.5932703614234924
Recall :  0.7389705777168274
TruePositives :  1005.0
TrueNegatives :  6591.0
FalsePositives :  689.0
FalseNegatives :  355.0
